<a href="https://colab.research.google.com/github/frankausberlin/lazy_test/blob/main/lazystudentnotebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

<table><tr><td align=center><font size=20>🧭</font></td><td>

**Unit tests are so good** - it's really worth using them early on. Unit tests make sense<br>especially in component-oriented software development.

1. A unit test should cover all use cases of a software. In particular, borderline cases<br>and extreme situations should be covered by test cases at an early stage.

2. By applying it consistently, you will achieve a high level of robustness in the software<br>you develop and will always have a stable (releaseable) version.

3. They reveal weaknesses in software design early on, especially in complex inheritance<br>hierarchies. In fact, this worked very well for me in the refactoring process. In an iterative<br>process, the basic structure of the software was changed several times - each time<br>triggered by insights I gained while designing test cases.

In the notebook you can enable and disable testing as you wish.
</td></tr></table>

# ButtonBox

To make optimal use of an area to display selection options. The Flex Wrap layout is well suited for this.


In [1]:
#@markdown <font size='+2' color='#005F6A'>**ButtonBox**</font><br>
#@markdown * An alternative for the Tab widget for selecting **large amount of items**
#@markdown * Uses **Buttons** and a **Box** with **flex wrap layout**.
#@markdown * Provide a **mechanism for select, append and delete** an item .
#@markdown <table><tr><td><font size='+2'>
#@markdown
#@markdown ```python
#@markdown def showSelection (bbox):
#@markdown     try:    textarea.value = plain (render_doc(bbox.button.tooltip))
#@markdown     except: textarea.value = 'error calling render_doc with '+bbox.button.tooltip
#@markdown #
#@markdown textarea  = Textarea  (layout=Layout(width='1000px', height='300px'))
#@markdown buttonbox = ButtonBox (__builtins__.__dict__.keys(), clicker=showSelection)
#@markdown display (buttonbox.widget, textarea)
#@markdown ```
from ipywidgets import Button, Box, Layout, Textarea

class ButtonBox ():
  def __init__ (self, descriptions, clicker=None, maxchar=60, color='powderblue'):
    # remember stuff - list to set
    self.descriptions = descriptions if len (descriptions) == len (set (descriptions)) else set (descriptions)
    self.clicker, self.maxchar, self.color, self.position, self.button = clicker, maxchar, color, -1, None

    # make buttons
    self.buttons = [Button (description = i if len (i) <= maxchar else f'{i[:(maxchar-3)]}...',
                            layout      = Layout(width='auto', height='21px'),
                            tooltip     = f'{i}')
                    for i in descriptions]

    # put them in a box / bind event
    self.widget = Box (layout   = Layout (display='flex', flex_flow='wrap'),
                       children = self.buttons)
    for button in self.buttons: button.on_click (self._clicker)

  # An alternative for the Tab widget
  def _clicker (self, b):
    # search clicked button and remember
    self.position, self.button = [(i,but) for i, but in enumerate(self.buttons) if b == but][0]

    # unselect (color) all buttons and select new (list of colors here: https://www.quackit.com/css/css_color_codes.cfm)
    self.unselect()
    self.buttons[self.position].style={'button_color':self.color}

    # fire event
    if self.clicker: self.clicker (self)

  def append (self, description, select=True):
    # add new selctor button at the end
    b = Button (description = description if len (description) <= self.maxchar else f'{description[:(self.maxchar-3)]}...',
                layout      = Layout(width='auto', height='21px'),
                tooltip     = f'{description}')
    self.buttons.append(b)
    self.widget.children = [*self.widget.children, b]

    # select new button / bind event
    if select:
      self.button, self.position = b, len(self.buttons) - 1
      self._clicker (b)
    b.on_click (self._clicker)

  def remove (self, position=None):
    # remove selected if no position given
    if not position: position = self.position
    if not position or not position < len(self.buttons) or position < 0: return

    # clean up
    self.widget.children = [*self.widget.children[:position], *self.widget.children[position+1:]]
    self.buttons = [*self.buttons[:position], *self.buttons[position+1:]]
    self.button, self.position = None, -1

  def unselect (self):
    # unselect all buttons
    for b in self.buttons: b.style={'button_color':None}

# ___________________________________________________________
#|______________________hello_component______________________|

from ipywidgets   import Layout, Textarea, Button
from pydoc        import plain, render_doc

def showSelection (bbox):
  try:    textarea.value = plain (render_doc(bbox.button.tooltip))
  except: textarea.value = 'error calling render_doc with '+bbox.button.tooltip

textarea  = Textarea  (layout=Layout(width='1000px', height='300px'))
buttonbox = ButtonBox (__builtins__.__dict__.keys(), clicker=showSelection)
display   (buttonbox.widget, textarea)

(del_but := Button (description="delete")).on_click (lambda b: buttonbox.remove ())
(app_but := Button (description="append")).on_click (lambda b: buttonbox.append ('new button'))
display  (app_but, del_but)

# alternative
# def showSelection (bbox):
#     try:    textarea.value = str (globals()[bbox.button.tooltip])
#     except: textarea.value = 'opps'
# #
# textarea  = Textarea  (layout=Layout(width='1000px', height='300px'))
# buttonbox = ButtonBox (globals ().keys(), clicker=showSelection)
# display (buttonbox.widget, textarea)


Box(children=(Button(description='__name__', layout=Layout(height='21px', width='auto'), style=ButtonStyle(), …

Textarea(value='', layout=Layout(height='300px', width='1000px'))

Button(description='append', style=ButtonStyle())

Button(description='delete', style=ButtonStyle())

In [2]:
#@markdown <font size='-1' color='#005F6A'>**UnitTest ButtonBox**</font><br>

import unittest
from IPython.display import clear_output
only_register_test_ButtonBox = True # @param {type:"boolean"}
register_and_run_test_ButtonBox = False # @param {type:"boolean"}

if only_register_test_ButtonBox or register_and_run_test_ButtonBox:
  class Test_ButtonBox (unittest.TestCase):
    def setUp (self):
      self.data       = ['aaaa_item_a', 'bbbb_item_b', 'cccc_item_c']
      self.maxchar    = 8
      self.buttonBox  = ButtonBox (self.data, maxchar=self.maxchar, color='powderblue')

    def sub_allUnselect (self):
      with self.subTest(msg='check if all buttons are unselected'):
        # test all buttons
        for b in self.buttonBox.buttons:
          self.assertEqual (b.style.button_color, None)

      with self.subTest(msg='check no button marked as selected'):
        self.assertEqual (self.buttonBox.button, None)
        self.assertEqual (self.buttonBox.position, -1)

    def sub_selectionCorrect (self, bnr):
      with self.subTest(msg=f'check if button {bnr} selected correct'):
        # test all buttons
        for i in range (3):
          # check if wrong not selected
          if i != bnr:
            self.assertNotEqual (self.buttonBox.button, self.buttonBox.buttons[i])
            self.assertEqual (self.buttonBox.buttons[i].style.button_color, None)
          # check if correct selected
          else:
            self.assertEqual (self.buttonBox.button, self.buttonBox.buttons[i])
            self.assertEqual (self.buttonBox.buttons[i].style.button_color, 'powderblue')

    def test_ButtonBox_selection (self):
      # test init state
      self.sub_allUnselect ()

      # click every button
      for bnr in range (3):
        self.buttonBox._clicker (self.buttonBox.buttons[bnr])
        self.sub_selectionCorrect (bnr)

    def test_ButtonBox_maxchar_and_tooltip (self):
      self.assertEqual (self.buttonBox.buttons[0].description, 'aaaa_...')
      self.assertEqual (self.buttonBox.buttons[1].description, 'bbbb_...')
      self.assertEqual (self.buttonBox.buttons[2].description, 'cccc_...')
      self.assertEqual (self.buttonBox.buttons[0].tooltip, 'aaaa_item_a')
      self.assertEqual (self.buttonBox.buttons[1].tooltip, 'bbbb_item_b')
      self.assertEqual (self.buttonBox.buttons[2].tooltip, 'cccc_item_c')

    def test_ButtonBox_remove (self):
      self.buttonBox.remove (1)
      self.assertEqual (len(self.buttonBox.buttons),2)
      self.assertEqual (self.buttonBox.position, -1)
      self.assertEqual (self.buttonBox.button, None)

    def test_ButtonBox_append_with_select (self):
      self.buttonBox.append ('0123456789_test', select=True)
      self.assertEqual (len(self.buttonBox.buttons[-1].description),self.maxchar)
      self.assertEqual (len(self.buttonBox.buttons),4)
      self.sub_selectionCorrect (3)

    def test_ButtonBox_append_without_select (self):
      self.buttonBox.append ('0123456789_test', select=False)
      self.assertEqual (len(self.buttonBox.buttons[-1].description),self.maxchar)
      self.assertEqual (len(self.buttonBox.buttons),4)
      self.sub_allUnselect ()

if register_and_run_test_ButtonBox:
  result = unittest.main(argv=[""], verbosity=2, exit=False).result



# Selectable
Selectable is used to implement a selectable list of items. An item is represented by an HBox widget with a small button at the beginning for selection. The basic idea is to 'attach' your own widgets to an item for the list. In the application, for example, an HTML widget is used to provide links to YouTube videos.




In [3]:
""" Colab Code-Snippets
It uses colab forms to make a top-level setting of the code snippet (optional) and give
a little description. You can run the cell to execute the code snippet.
"""
#@markdown <font size='+2' color='#005F6A'>**Selectable**</font><br>
#@markdown * Provides a mechanism for selecting items with either **radio** or **multi-select** behavior.
#@markdown * Uses **Buttons** to represent selectable items.
#@markdown * Allows for **custom behavior** on first selection and subsequent selections.
#@markdown <table><tr><td><font size='+2'>
#@markdown
#@markdown ```python
#@markdown class MySelectable (Selectable):
#@markdown   def setInitState (self):            self.setItemWidget (Label (value="initial state"))
#@markdown   def onItemSelect (self, posList):   print ('Your selection:',posList)
#@markdown   def __init__(self, item, items, behave='radio'):
#@markdown     super().__init__(item, items, behave=behave, selector=self.onItemSelect)
#@markdown #
#@markdown lst = []
#@markdown display (HBox (children = [MySelectable('a', lst, behave='multi').widget,
#@markdown                            MySelectable('b', lst, behave='multi').widget]) )
#@markdown ```
#@markdown </td></tr></table>
from ipywidgets import Button, HTML, HBox, VBox, Label

class Selectable:
  """A class representing a selectable item with customizable behavior.
  This code snippet demonstrates the use of a custom selectable widget class
  in a Jupyter notebook. It allows for the selection of items using either
  radio button behavior or multi-select behavior.

  Attributes
  ----------
  * item: obj
    The item can be a object or a string or something else
  * items: list
    A list with Selectable objects.
  * behave: str
    The behavior type ('multi' or 'radio' or 'radiox') for selection.
  * onSelect: function | default: None
    A callback function triggered upon selection.
  * widget: HBox
    The widget representing the selectable item.

  Methods
  -------
  * doBehavior() | -> none
    Executes the selection behavior based on the specified type.
  * _onSelectorClick(b: Button) | -> none
    Handles the click event for the selector button.
  * setItemWidget(widget) | -> none
    Sets the widget to display for the item.
  * setInitState() | -> none
    Initializes the state of the item (to be implemented in subclasses).
  * onFirstSelect(posList) | -> none
    Defines behavior on the first selection (to be implemented in subclasses).

  Behavior
  -------
  * You can set the behavior to 'radio' or 'multi' (radio button or multi selection).
  * On creation it calls the setInitState abstract function where you can build your
    own widget (e.g. HTML-object) and register it with setItemWidget.
  * When you select an item the abstract function onFirstSelect is called so you can
    set your widget in a 'working' state.
  * Every selection triggers the callback function onSelect if set.

  Use cases
  ---------
  * Make your own class, inherit from Selectable and implement the onFirstSelect
    and setInitState function.
  * To react on a select you can give a listener in the super constructor call.
  * To combine several selectable objects to a set you have to give the same list
    on construction.
  * Display an item with the .widget attribute.

  Example
  -------
  class MySelectable (Selectable):
    def setInitState (self):            self.setItemWidget (Label (value="initial state"))
    def onFirstSelect (self, posList):  self.itemWidget.value = "initialized "+self.item
    def onSelection (self, posList):    print ('Your selection:',posList)
    def __init__(self, item, items, behave='radio'):
      super().__init__(item, items, behave, self.onSelection)

  # example: two sets of Selectable objects (two lines radio button / one line multi select)
  a, b = [] , []
  #
  display (HBox (children = [MySelectable('a', a).widget, MySelectable('b', a).widget, MySelectable('c', a).widget] ))
  display (HBox (children = [MySelectable('d', a).widget, MySelectable('e', a).widget, MySelectable('f', a).widget] ))
  display (HTML(value='<hr>'))
  #
  display (HBox (children = [MySelectable('a', b, 'multi').widget, MySelectable('b', b, 'multi').widget,
                             MySelectable('c', b, 'multi').widget, MySelectable('d', b, 'multi').widget,
                             MySelectable('e', b, 'multi').widget, MySelectable('f', b, 'multi').widget] ))

  """
  def __init__(self, item, items, behave, selector=None):
    """Initializes the class with given parameters.
    set attributes, generate widgets, bind events and set initial state.
    """
    self.isSelected, self.item, self.items, self.behave = False, item, items, behave
    self.bu_selector = Button(style={'button_color': '#99bfc3'}, layout={'width': '22px', 'height': '22px'})
    self.widget = HBox(children=[self.bu_selector], layout={'min_height': '24px', 'overflow': 'hidden'})
    self.items.append(self)
    self.bu_selector.on_click(self.select)
    self.setInitState()
    self.selector = selector


  def doBehavior (self):
    """Executes the selection behavior based on the specified type.
    """
    self.posList = []

    if self.behave == 'radiox':
      state         = self.items[self.lastSelect].isSelected
      for item in self.items: item.bu_selector.style.button_color, item.isSelected = '#99bfc3', False
      if state:     self.items[self.lastSelect].bu_selector.style.button_color, self.items[self.lastSelect].isSelected = '#99bfc3', False
      else:         self.items[self.lastSelect].bu_selector.style.button_color, self.items[self.lastSelect].isSelected = '#005F6A', True
      self.posList  = [p for p in range (len(self.items)) if self.items[p].isSelected]

    elif self.behave == 'radio':
      for item in self.items: item.bu_selector.style.button_color, item.isSelected = '#99bfc3', False
      self.bu_selector.style.button_color, self.isSelected, self.posList = '#005F6A', True, [self.lastSelect]

    elif self.behave == 'multi':
      state         = self.items[self.lastSelect].isSelected
      if state:     self.items[self.lastSelect].bu_selector.style.button_color, self.items[self.lastSelect].isSelected = '#99bfc3', False
      else:         self.items[self.lastSelect].bu_selector.style.button_color, self.items[self.lastSelect].isSelected = '#005F6A', True
      self.posList  = [p for p in range (len(self.items)) if self.items[p].isSelected]

    else: raise Exception('unknown behavior: '+self.behave)

  def setItemWidget (self, widget):
    """Sets the widget to display for the item.
    """
    self.itemWidget, self.widget.children = widget, (*self.widget.children, widget)

  def setInitState (self): raise NotImplementedError
  def onSelection (self, posList): raise NotImplementedError

  def select (self, b=None):
    """Handles the click event for the selector button.
    """
    for buttonPos in range (len(self.items)):
      if self.items[buttonPos].bu_selector == self.bu_selector: break
    if buttonPos < len(self.items):   self.lastSelect = buttonPos
    else:                             self.lastSelect = -1
    self.doBehavior ()
    if self.selector: self.selector (self.posList)


# ___________________________________________________________
#|______________________hello_component______________________|
class MySelectable (Selectable):
  def setInitState (self):            self.setItemWidget (Label (value="initial state"))
  def onItemSelect (self, posList):   print ('\rYour selection:',posList, end='')
  def __init__(self, item, items, behave='radio'):
    super().__init__(item, items, behave, self.onItemSelect)

# example: two sets of Selectable objects (two lines radio button / one line multi select)
a, b, c = [] , [], []
#
display (HTML(value='<font size=+1>behave: radio'))
display (HBox (children = [MySelectable('a', a).widget, MySelectable('b', a).widget, MySelectable('c', a).widget,
                           MySelectable('d', a).widget, MySelectable('e', a).widget, MySelectable('f', a).widget] ))
#
display (HTML(value='<hr><font size=+1>behave: multi'))
display (HBox (children = [MySelectable('a', b, 'multi').widget, MySelectable('b', b, 'multi').widget,
                           MySelectable('c', b, 'multi').widget, MySelectable('d', b, 'multi').widget,
                           MySelectable('e', b, 'multi').widget, MySelectable('f', b, 'multi').widget] ))

#
display (HTML(value='<hr><font size=+1>behave: radiox'))
display (HBox (children = [MySelectable('a', c, 'radiox').widget, MySelectable('b', c, 'radiox').widget,
                           MySelectable('c', c, 'radiox').widget, MySelectable('d', c, 'radiox').widget,
                           MySelectable('e', c, 'radiox').widget, MySelectable('f', c, 'radiox').widget] ))


HTML(value='<font size=+1>behave: radio')

HTML(value='<hr><font size=+1>behave: multi')

HTML(value='<hr><font size=+1>behave: radiox')

In [4]:
#@markdown <font size='-1' color='#005F6A'>**UnitTest Selectable**</font><br>

import unittest
from IPython.display import clear_output
only_register_test_Selectable = True # @param {type:"boolean"}
register_and_run_test_Selectable = True # @param {type:"boolean"}

if only_register_test_Selectable or register_and_run_test_Selectable:
  class Test_Selectable (unittest.TestCase):
    def setUp (self):
      # class for test inherits from Selectable
      class MySelectable (Selectable):
        def setInitState (self):            self.setItemWidget (Label (value="initial state"))
        def onItemSelect (self, posList):   pass
        def __init__(self, item, items, behave='radio'): super().__init__(item, items, behave, self.onItemSelect)

      # example sets a, b, c for radio, multi, radiox
      self.a, self.b, self.c, myPositions = [], [] , [], lambda l: [i for i, s in enumerate(l) if s.isSelected]

      MySelectable('a1', self.a,  'radio'), MySelectable('a2', self.a,  'radio'), MySelectable('a3', self.a,  'radio')
      MySelectable('b1', self.b,  'multi'), MySelectable('b2', self.b,  'multi'), MySelectable('b3', self.b,  'multi')
      MySelectable('c1', self.c, 'radiox'), MySelectable('c2', self.c, 'radiox'), MySelectable('c3', self.c, 'radiox')

    def selectedPositions (items): return [pos for pos, i in enumerate(items) if i.isSelected]

    def test_Selectable_generation (self):
      # test generation
      self.assertEqual (len(self.a), 3)
      self.assertEqual (len(self.b), 3)
      self.assertEqual (len(self.c), 3)
      self.assertEqual ([i.itemWidget.value for i in self.a], ['initial state', 'initial state', 'initial state'])
      self.assertEqual ([i.itemWidget.value for i in self.b], ['initial state', 'initial state', 'initial state'])
      self.assertEqual ([i.itemWidget.value for i in self.c], ['initial state', 'initial state', 'initial state'])

    def test_Selectable_behave_radio (self):
      # simulate button clicks and test effects
      self.a[0].select(self.a[0].widget) # click once
      self.assertEqual ([0], Test_Selectable.selectedPositions (self.a))
      self.assertEqual ([self.a[0].isSelected, self.a[1].isSelected, self.a[2].isSelected], [True,False,False])
      self.a[0].select(self.a[0].widget) # click two times
      self.assertEqual ([0], Test_Selectable.selectedPositions (self.a))
      self.assertEqual ([self.a[0].isSelected, self.a[1].isSelected, self.a[2].isSelected], [True,False,False])
      self.a[1].select(self.a[1].widget) # click once
      self.assertEqual ([1], Test_Selectable.selectedPositions (self.a))
      self.assertEqual ([self.a[0].isSelected, self.a[1].isSelected, self.a[2].isSelected], [False,True,False])
      self.a[1].select(self.a[1].widget) # click two times
      self.assertEqual ([1], Test_Selectable.selectedPositions (self.a))
      self.assertEqual ([self.a[0].isSelected, self.a[1].isSelected, self.a[2].isSelected], [False,True,False])
      self.a[2].select(self.a[2].widget) # click once
      self.assertEqual ([2], Test_Selectable.selectedPositions (self.a))
      self.assertEqual ([self.a[0].isSelected, self.a[1].isSelected, self.a[2].isSelected], [False,False,True])
      self.a[2].select(self.a[2].widget) # click two times
      self.assertEqual ([2], Test_Selectable.selectedPositions (self.a))
      self.assertEqual ([self.a[0].isSelected, self.a[1].isSelected, self.a[2].isSelected], [False,False,True])

    def test_Selectable_behave_multi (self):
      # simulate button clicks and test effects
      self.b[0].select(b[0].widget)
      self.assertEqual ([0],Test_Selectable.selectedPositions(self.b))
      self.assertEqual ([self.b[0].isSelected,self.b[1].isSelected,self.b[2].isSelected], [True,False,False])
      self.b[1].select(self.b[1].widget)
      self.assertEqual ([0,1],Test_Selectable.selectedPositions(self.b))
      self.assertEqual ([self.b[0].isSelected,self.b[1].isSelected,self.b[2].isSelected], [True,True,False])
      self.b[2].select(self.b[2].widget)
      self.assertEqual ([0,1,2],Test_Selectable.selectedPositions(self.b))
      self.assertEqual ([self.b[0].isSelected,self.b[1].isSelected,self.b[2].isSelected], [True,True,True])
      self.b[1].select(self.b[1].widget)
      self.assertEqual ([0,2],Test_Selectable.selectedPositions(self.b))
      self.assertEqual ([self.b[0].isSelected,self.b[1].isSelected,self.b[2].isSelected], [True,False,True])
      self.b[2].select(self.b[2].widget)
      self.assertEqual ([0],Test_Selectable.selectedPositions(self.b))
      self.assertEqual ([self.b[0].isSelected,self.b[1].isSelected,self.b[2].isSelected], [True,False,False])

    def test_Selectable_behave_radiox (self):
      # simulate button clicks and test effects
      self.c[0].select(self.c[0].widget) # click once
      self.assertEqual ([0], Test_Selectable.selectedPositions (self.c))
      self.assertEqual ([self.c[0].isSelected, self.c[1].isSelected, self.c[2].isSelected], [True,False,False])
      self.c[0].select(self.c[0].widget) # click two times
      self.assertEqual ([], Test_Selectable.selectedPositions (self.c))
      self.assertEqual ([self.c[0].isSelected, self.c[1].isSelected, self.c[2].isSelected], [False,False,False])
      self.c[1].select(self.c[1].widget) # click once
      self.assertEqual ([1], Test_Selectable.selectedPositions (self.c))
      self.assertEqual ([self.c[0].isSelected, self.c[1].isSelected, self.c[2].isSelected], [False,True,False])
      self.c[1].select(self.c[1].widget) # click two times
      self.assertEqual ([], Test_Selectable.selectedPositions (self.c))
      self.assertEqual ([self.c[0].isSelected, self.c[1].isSelected, self.c[2].isSelected], [False,False,False])
      self.c[2].select(self.c[2].widget) # click once
      self.assertEqual ([2], Test_Selectable.selectedPositions (self.c))
      self.assertEqual ([self.c[0].isSelected, self.c[1].isSelected, self.c[2].isSelected], [False,False,True])
      self.c[2].select(self.c[2].widget) # click two times
      self.assertEqual ([], Test_Selectable.selectedPositions (self.c))
      self.assertEqual ([self.c[0].isSelected, self.c[1].isSelected, self.c[2].isSelected], [False,False,False])

if register_and_run_test_Selectable:
  result = unittest.main(argv=[""], verbosity=2, exit=False).result


test_ButtonBox_append_with_select (__main__.Test_ButtonBox.test_ButtonBox_append_with_select) ... ok
test_ButtonBox_append_without_select (__main__.Test_ButtonBox.test_ButtonBox_append_without_select) ... ok
test_ButtonBox_maxchar_and_tooltip (__main__.Test_ButtonBox.test_ButtonBox_maxchar_and_tooltip) ... ok
test_ButtonBox_remove (__main__.Test_ButtonBox.test_ButtonBox_remove) ... ok
test_ButtonBox_selection (__main__.Test_ButtonBox.test_ButtonBox_selection) ... ok
test_Selectable_behave_multi (__main__.Test_Selectable.test_Selectable_behave_multi) ... ok
test_Selectable_behave_radio (__main__.Test_Selectable.test_Selectable_behave_radio) ... ok
test_Selectable_behave_radiox (__main__.Test_Selectable.test_Selectable_behave_radiox) ... ok
test_Selectable_generation (__main__.Test_Selectable.test_Selectable_generation) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.103s

OK
